In [44]:
class genericField:
  def __init__(self, bbox=(0, 0, 0, 0),fieldType=None):
    self.fieldType=fieldType
    self.BoundingBox = {
            "Left": bbox[0],  
            "Top": bbox[1], 
            "Width": bbox[2],
            "Height": bbox[3]
            }
  def __str__(self):
    return str(self._dict_)
  def __repr__(self):
    return self._str_()

class dateField(genericField):
  def __init__(self, bbox=(0, 0, 0, 0)):
    super(dateField, self).__init__(bbox,"date")
    self.date={
        "subFields" : 0,       #3 for dd/mm/yyyy
        "side" : 0,
        "count" : []          #[2,2,4] for dd/mm/yyyy
    }

class blocksField(genericField):
  def __init__(self,bbox=(0, 0, 0, 0)):
    super(blocksField, self).__init__(bbox,"blocksField")
    self.blocks={
        "side":0, # side length of each block
        "count":0 # number of blocks
    }

In [45]:
import cv2
import numpy as np
from PIL import Image
        
img1 = cv2.imread('form_out.jpeg')
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img = cv2.GaussianBlur(img,(1,3),0)
img_bin,ab = cv2.threshold(img,220,255,cv2.THRESH_BINARY )
cv2.imwrite("Thresholded_img.jpg",ab)
binary_map = 255-ab

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_map, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result
#cv2.imwrite("Thresholded_img.jpg",result)
img_bin,result = cv2.threshold(result,248,255,cv2.THRESH_BINARY )
kernel = np.ones((3, 3), np.uint8)   
result = cv2.erode(result, kernel,iterations = 4) 
result = cv2.GaussianBlur(result,(3,3),0)
cv2.imwrite("Image_bin_new.jpg",result)
boxes = list()
contours, hierarchy = cv2.findContours(result, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if h<30 and abs(w-h)<6 and w<30:
          new_img = img[y:y+h, x:x+w]
          cv2.rectangle(img1, (x, y), (x+w, y+h), (255, 0, 0), 1)
          boxes.append((x,y,w,h))
cv2.imwrite("RESULT1.jpg",img1)
cv2.imshow("IMAGE",img1)
cv2.waitKey(0)

-1

In [46]:
boxes.sort(key = lambda x:x[1])
print(boxes)

[(731, 184, 22, 17), (704, 184, 20, 17), (675, 184, 20, 17), (649, 184, 18, 17), (615, 184, 20, 17), (586, 184, 20, 17), (551, 184, 19, 17), (523, 184, 20, 17), (375, 184, 19, 17), (490, 232, 20, 17), (461, 232, 20, 17), (433, 232, 21, 17), (405, 232, 20, 17), (367, 232, 20, 17), (340, 232, 20, 17), (943, 363, 19, 18), (890, 363, 19, 17), (943, 415, 19, 18), (890, 415, 19, 17), (890, 465, 19, 18), (943, 466, 19, 17), (507, 548, 18, 18), (453, 548, 19, 18), (963, 551, 14, 12), (943, 640, 19, 17), (888, 641, 20, 16), (736, 756, 20, 17), (708, 756, 20, 17), (680, 756, 20, 17), (651, 756, 20, 17), (623, 756, 21, 17), (595, 756, 20, 17), (566, 756, 20, 17), (538, 756, 20, 17), (510, 756, 20, 17), (482, 756, 20, 17), (453, 756, 20, 17), (425, 756, 21, 17), (397, 756, 20, 17), (317, 756, 21, 17), (289, 756, 20, 17), (260, 756, 20, 17), (232, 756, 20, 17), (204, 756, 20, 17), (176, 756, 20, 17), (147, 756, 20, 17), (119, 756, 20, 17), (91, 756, 20, 17), (62, 756, 20, 17), (34, 756, 21, 17), (8

In [47]:
li1 = list()
i = 0 
while i < len(boxes)-1:
    temp = list()
    while i < len(boxes)-1 and boxes[i][1]==boxes[i+1][1]:
        temp.append(boxes[i])
        i+=1
    temp.append(boxes[i])
    li1.append(temp)
    i+=1
print(li1)

[[(731, 184, 22, 17), (704, 184, 20, 17), (675, 184, 20, 17), (649, 184, 18, 17), (615, 184, 20, 17), (586, 184, 20, 17), (551, 184, 19, 17), (523, 184, 20, 17), (375, 184, 19, 17)], [(490, 232, 20, 17), (461, 232, 20, 17), (433, 232, 21, 17), (405, 232, 20, 17), (367, 232, 20, 17), (340, 232, 20, 17)], [(943, 363, 19, 18), (890, 363, 19, 17)], [(943, 415, 19, 18), (890, 415, 19, 17)], [(890, 465, 19, 18)], [(943, 466, 19, 17)], [(507, 548, 18, 18), (453, 548, 19, 18)], [(963, 551, 14, 12)], [(943, 640, 19, 17)], [(888, 641, 20, 16)], [(736, 756, 20, 17), (708, 756, 20, 17), (680, 756, 20, 17), (651, 756, 20, 17), (623, 756, 21, 17), (595, 756, 20, 17), (566, 756, 20, 17), (538, 756, 20, 17), (510, 756, 20, 17), (482, 756, 20, 17), (453, 756, 20, 17), (425, 756, 21, 17), (397, 756, 20, 17), (317, 756, 21, 17), (289, 756, 20, 17), (260, 756, 20, 17), (232, 756, 20, 17), (204, 756, 20, 17), (176, 756, 20, 17), (147, 756, 20, 17), (119, 756, 20, 17), (91, 756, 20, 17), (62, 756, 20, 17), 

In [50]:
blocks = list()
for i in range(len(li1)):
    dif = list()
    li1[i].sort()
    for j in range(len(li1[i])-1):        
       dif.append(li1[i][j+1][0]-li1[i][j][0])
    blocks.append(dif)
print(blocks)

[[148, 28, 35, 29, 34, 26, 29, 27], [27, 38, 28, 28, 29], [53], [53], [], [], [54], [], [], [], [28, 29, 28, 28, 29, 28, 28, 28, 29, 28, 80, 28, 28, 29, 28, 28, 28, 29, 28, 28, 29, 28, 28], [28, 29, 28, 28, 29, 28, 28, 28, 29, 28, 190, 28, 29, 28, 28, 28, 29, 28, 28, 29, 27, 29, 29], [28, 29, 28, 28, 29, 28, 28, 28, 29, 28, 28, 162, 28, 29, 28, 28, 28, 29, 28, 28, 29, 27], [54], [55], [30, 23, 549, 55], []]
